In [1]:
import sys
print(sys.version)
import torch
import torch.nn as nn

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import OrderedDict

# Import PyTorch

from torch.autograd import Variable

from torch.autograd import Function # import Function to create custom activations
from torch.nn.parameter import Parameter # import Parameter to create custom activations with learnable parameters
from torch import optim # import optimizers for demonstrations
import torch.nn.functional as F # import torch functions
from torchvision import datasets, transforms # import transformations to use for demo

3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]


In [172]:
def tensor_inf_to_zero(bad_tensor):
    bad_tensor[bad_tensor == float('inf')] = 0
    return bad_tensor


# TODO: tests
def calc_forces(state, prints = False):

    aux1 = torch.tensor(([1.,0.],[0.,1.])) # used to transform state from [N_rows*2] to [N_rows*(2*N_rows)]
    '''
        e.g. : state   [[1,  0]
                        [2,  1]
                        [-1,-1]
                        [0, -1]]
        
        new state_concated:         
                       [[1,  0, 1,  0, 1,  0, 1,  0]
                        [2,  1, 2,  1, 2,  1, 2,  1]
                        [-1,-1, -1,-1, -1,-1, -1,-1]
                        [0, -1, 0, -1, 0, -1, 0, -1]]
                        

    '''

    for i in range(0,state.shape[0]-1):
        aux1 = torch.cat((aux1,torch.tensor(([1.,0.],[0.,1.]))),dim=1)
    
    
    aux1.requires_grad_(True)
    aux1.retain_grad()
    
    state_concated = state.matmul(aux1).requires_grad_(True)
    state_concated.retain_grad()
    
    state_concated_t = state.view(1,-1).requires_grad_(True)
    for i in range(0,state.shape[0]-1):
        state_concated_t = torch.cat([state_concated_t,state.view(1,-1)]).requires_grad_(True)
        
        '''    state_concated_t tensor(
        [[ 1.,  0.,  2.,  1., -1., -1.,  0., -1.],
         [ 1.,  0.,  2.,  1., -1., -1.,  0., -1.],
         [ 1.,  0.,  2.,  1., -1., -1.,  0., -1.],
         [ 1.,  0.,  2.,  1., -1., -1.,  0., -1.]]
        '''
    
    state_concated_t.requires_grad_(True)

    delta_pose = (state_concated_t - state_concated).requires_grad_(True)
    delta_pose.retain_grad()
    print ("delta_pose ", delta_pose)
    
    auxullary = torch.zeros(state_concated.shape[0], state_concated.shape[1])
    auxullary.retain_grad()
    for i in range(state_concated.shape[0]):

        auxullary[i,2*i] = 1.
        auxullary[i,2*i+1] = 1.
    print ("auxullary ", auxullary)
    ''' used to calc x+y of each agent pose
        auxullary  tensor([
            [1., 1., 0., 0., 0., 0., 0., 0.],
            [0., 0., 1., 1., 0., 0., 0., 0.],
            [0., 0., 0., 0., 1., 1., 0., 0.],
            [0., 0., 0., 0., 0., 0., 1., 1.]])
    '''

#     outputs = auxullary*delta_pose.requires_grad_(True)
#     outputs.retain_grad()
#     print ("outputs ", outputs) 
#     s_d = torch.sum(outputs, dim=0).requires_grad_(True)
#     s_d.retain_grad()
#     print ("s_d ", s_d) 
#     st2 = ((delta_pose - s_d)**2).requires_grad_(True)
    dist_squared = ((delta_pose)**2).requires_grad_(True)
    dist_squared.retain_grad()
    print ("dist_squared ", dist_squared) 
    
    # used to calc delta_x**2 +delta_y**2 of each agent
    aux = auxullary.t()
    aux.retain_grad()
    # sqrt(delta_x**2 +delta_y**2) -> distance
    dist = torch.sqrt(dist_squared.matmul(aux)).requires_grad_(True) ## aka distance

    print ("dist ", dist)
#     dist.retain_grad()

    # force `amplitude` = 1/d
    force_amplitude = tensor_inf_to_zero(1/dist).requires_grad_(True) ## aka 1/distance
    force_amplitude.retain_grad()
    
    print ("force_amplitude ", force_amplitude)
    force_amplitude.backward(dist)
    print ("dist.grad ",dist.grad)

    # f ~ 1/[d] * ([delta_pose]/[dist])
    force = force_amplitude.matmul(auxullary) * delta_pose/dist.matmul(auxullary)

    force[force != force] = 0
    force.requires_grad_(True)
    force.retain_grad()
    print ("force ", force)
    force.backward(state_concated)
    print ("state_concated.grad ", state_concated.grad)
    print ("state_concated.requires_grad ", state_concated.requires_grad)
    print ("force.requires_grad ", force.requires_grad)
    aux1 = aux1.t()
    aux1.requires_grad_(True)
    aux1.retain_grad()
    force = force.matmul(aux1).requires_grad_(True)
    force.retain_grad()
    
    
    return force

In [173]:
class LinearFunction(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)

        output = calc_forces(input)
        print (output)
        output.retain_grad()
        output.backward(input)
        gr = input.grad 
        print (gr)
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.

        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.

        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]: 
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias    

    
class Linear(nn.Module):
    def __init__(self, input_features, output_features, bias=True):
        super(Linear, self).__init__()
        self.input_features = input_features
        self.output_features = output_features

        # nn.Parameter is a special kind of Tensor, that will get
        # automatically registered as Module's parameter once it's assigned
        # as an attribute. Parameters and buffers need to be registered, or
        # they won't appear in .parameters() (doesn't apply to buffers), and
        # won't be converted when e.g. .cuda() is called. You can use
        # .register_buffer() to register buffers.
        # nn.Parameters require gradients by default.

        self.weight = nn.Parameter(torch.Tensor(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_features))
        else:
            # You should always register all possible parameters, but the
            # optional ones can be None if you want.
            self.register_parameter('bias', None)

        # Not a very smart way to initialize weights
        self.weight.data.uniform_(-0.1, 0.1)
        if bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)

    def forward(self, input):
        # See the autograd section for explanation of what happens here.
        return LinearFunction.apply(input, self.weight, self.bias)

    def extra_repr(self):
        # (Optional)Set the extra information about this module. You can test
        # it by printing an object of this class.
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
    
l  = Linear(4,2, bias = None)

data = torch.randn((4, 2),requires_grad=True)
data = torch.tensor(([1.,  0],[2,  1],[-1,-1],[0, -1]))
res = l(data)
# print (data)
# print ()
# print (res)

delta_pose  tensor([[ 0.,  0.,  1.,  1., -2., -1., -1., -1.],
        [-1., -1.,  0.,  0., -3., -2., -2., -2.],
        [ 2.,  1.,  3.,  2.,  0.,  0.,  1.,  0.],
        [ 1.,  1.,  2.,  2., -1.,  0.,  0.,  0.]], requires_grad=True)
auxullary  tensor([[1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1.]])
dist_squared  tensor([[0., 0., 1., 1., 4., 1., 1., 1.],
        [1., 1., 0., 0., 9., 4., 4., 4.],
        [4., 1., 9., 4., 0., 0., 1., 0.],
        [1., 1., 4., 4., 1., 0., 0., 0.]], requires_grad=True)
dist  tensor([[0.0000, 1.4142, 2.2361, 1.4142],
        [1.4142, 0.0000, 3.6056, 2.8284],
        [2.2361, 3.6056, 0.0000, 1.0000],
        [1.4142, 2.8284, 1.0000, 0.0000]], requires_grad=True)
force_amplitude  tensor([[0.0000, 0.7071, 0.4472, 0.7071],
        [0.7071, 0.0000, 0.2774, 0.3536],
        [0.4472, 0.2774, 0.0000, 1.0000],
        [0.7071, 0.3536, 1.0000, 0.0000]], req